## G08:   Óscar Morujo Fernández ,  Gorka Silva Ramón

# Parte 2. Recuperación de información

## Apartado a)

**Empezaremos utilizando una representación de la bolsa de palabras con las frecuencias de cada una en cada mensaje**

Cargamos los datos y los dividimos en train y test:

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
train_data = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
test_data = fetch_20newsgroups(subset='test')

print("Training texts:", len(train_data.data))
print("Test texts:", len(test_data.data))

Training texts: 11314
Test texts: 7532


Podemos observar 20 clases diferentes de mensajes :

In [3]:
print(len(train_data.target_names))
print(train_data.target_names)

20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


Creamos un dataframe que contiene los elementos de test. Por cada mensaje, tenemos la clase a la que pertenece y su id

In [4]:
import numpy as np
import pandas as pd

mensajes = {'Mensaje': test_data.data,
        'clase': test_data.target,
        'id': list(range(len(test_data.data)))}
df = pd.DataFrame(mensajes, columns=['Mensaje', 'clase','id'])
df.head()

,Mensaje,clase,id
0,From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. ...,7,0
1,From: Rick Miller <rick@ee.uwm.edu>\nSubject: ...,5,1
2,From: mathew <mathew@mantis.co.uk>\nSubject: R...,0,2
3,From: bakken@cs.arizona.edu (Dave Bakken)\nSub...,17,3
4,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,19,4


**A continuación vamos a crear nuestra bolsa de palabras con ayuda de CountVectorizer. Le pasamos el diccionario que lo cargamos desde "words.txt" y lo configuramos para que use las frecuencias de aparición de cada palabra(considerando solo monogramas):**

In [5]:
# Pasamos el fichero a una lista (una línea por item)
with open('Datos/words.txt') as f:
    dictionary = f.read().splitlines()

vectorizer = CountVectorizer(vocabulary=dictionary, stop_words='english',
                             binary=False,ngram_range=(1,1))
# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
train_vector_data=vectorizer.fit_transform(train_data.data)

In [6]:
test_vector_data=vectorizer.transform(test_data.data)

In [7]:
feature_names = vectorizer.get_feature_names()

print(len(feature_names))

466551


**Tomamos 3 mensajes de cada clase del conjunto de test (es decir, para cada tema)**

Usaremos cada uno de estos mensajes como consulta para recuperar los mensajes del conjunto de entrenamiento que mas se parezcan a la consulta.

Vamos a declarar una función que nos devuelva una lista que contenga en cada posición "i" los 3 mensajes seleccionados de la clase "i":

In [8]:
def mensajes_por_clase():
    lista_mensajes = []
    for i in range(0,len(test_data.target_names)):
        df2 = df[df['clase'] == i]#df2->mensajes de la clase i
        new=[]
        for j in range(0,3):#Seleccionamos los 3 primeros de cada clase
            new.append(df2.iloc[j,2])#Nos desplazamos por las filas con j sobre la columna 2(clase)
            
        lista_mensajes.append(new)

    return lista_mensajes

Obtenemos una lista que contiene 3 mensajes de cada clase (en nuestro caso, los primeros que se encuentren en el dataframe al aplicar un filtro para mostrar los datos por clases):

In [9]:
lista_mensajes = mensajes_por_clase()
lista_mensajes #Variable global, la usaremos más adelante

[[2, 14, 49],
 [9, 17, 20],
 [10, 15, 41],
 [24, 29, 33],
 [16, 32, 37],
 [1, 8, 11],
 [18, 21, 38],
 [0, 27, 28],
 [13, 42, 50],
 [36, 53, 69],
 [56, 61, 72],
 [26, 44, 48],
 [84, 109, 120],
 [5, 25, 52],
 [23, 34, 46],
 [6, 7, 87],
 [19, 58, 86],
 [3, 12, 22],
 [65, 66, 71],
 [4, 63, 73]]


Vemos como por ejemplo, para la clase 0 (atheism) hemos seleccionado los documentos con descriptores 2,14 y 49 del conjunto de test.

**A continuación definiremos dos funciones:**
    
    - calculate_cosine_similarity: calcula la distancia del conseno entre cada mensaje de consulta seleccionados previamente y todos los mensajes del conjunto de entrenamiento.
    
    - get_relevants(X): nos devuelve los X primeros términos más relevantes(ordenando de mayor a menor las similitudes entre dos mensajes)

In [10]:
from sklearn.metrics import pairwise

"""
Tenemos los mensajes seleccionados del conjunto de test que usaremos como consultas en la lista
"lista_mensajes" obtenida anteriormente.

Recibe como parametros los datos de test y de entrenamiento que contienen los elementos entre los
que se quiere calcular la distancia del coseno.

La funcion pairwise nos devuelve una matriz dispersa, ya que esta "pensada" para calcularlo entre
dos matrices, pero como nosotros lo usaremos para calcularlo entre un vector y una matriz, nos devolvera
una lista en la cual en cada posición tienes otra lista con 1 solo elemento(el vector de distancias),por 
eso nos quedaremos con la posición 0 de cada elemento de la matriz.

Asi devuelve una lista con el vector de similitudes entre cada uno de los 60 mensajes de consulta y todo el conjunto
de train
"""
def calculate_cosine_similarity(test_vector,train_vector):
    similarities=[]
    #Por cada clase tenemos 3 mensajes
    for clase in lista_mensajes:
        #Mensajes usados como consultas de la clase
        for id_mensaje in clase:
            similarities.append(pairwise.cosine_similarity(
                    test_vector[id_mensaje],train_vector)[0])
    return similarities


In [11]:
"""
Int X : nivel de exhaustividad
cosine_similarities : lista de distancias del coseno

LLama a getInfoRelevants (declarada mas abajo), que es una función auxiliar que nos ayudará a guardar en un string 
la información de la busqueda y recuperación

Devuelve una lista con los descriptores de los X primeros terminos más relevantes, además del string
con la info

"""
def get_relevants(X,cosine_similarities):
    
    array_posiciones=[]#ret id's de los recuperados
    resultados_consulta=""# ret info de los recuperados
    i = 0 #variable usada para identificar las 20 clases (de la 0 a la 19)
    contador=0 #variable para identificar cada uno de los 60 vectores de distancias del coseno
   
    #Por cada clase tenemos 3 mensajes usados como consulta
    for clase in lista_mensajes:  
        #Por cada clase tendremos 3 listas(1 por consulta) , y cada una de estas contienen
        # los id's de los X mensajes más relevantes recuperados de train  para esa consulta:
        clases =[]
        resultados_consulta += ("CONSULTAS CLASE "+  str(i) + '\n')
        resultados_consulta +=("=========================\n")
        for id_mensaje in clase:
            ## s contiene el array de distancias del conseno entre el  mensaje i_ésimo de la lista(contador)
            ##y el conjunto train
            s = cosine_similarities[contador] 
            contador+=1
            # best contiene los id de los X elementos con mayor valor de similitud del coseno
            best = s.argsort()[:-(X+1):-1] 
            clases.append(best)
            
            resultados_consulta+= getInfoRelevants(id_mensaje,best,s[best])
            
        #añadimos la matriz calculada anteriormente a la lista que finalmente retornaremos y la cual
        #contiene los mensajes recuperados de las 20 clases , es decir;con las 60 consultas.
        array_posiciones.append(clases)
        i+=1
        
    return array_posiciones,resultados_consulta

    


In [12]:
def getInfoRelevants(id_mensaje,docs_recuperados,distancias_coseno):
    resultados_consulta=""
    resultados_consulta +=("----Mensaje consulta nª " + str(id_mensaje))
    resultados_consulta +=('\n')
    resultados_consulta +=("--Id's documentos recuperados:")
    resultados_consulta +=('\n')
    resultados_consulta +=(str(docs_recuperados))
    resultados_consulta +=('\n')
    resultados_consulta +=("--Similitudes entre los documentos recuperados y la consulta:")
    resultados_consulta +=('\n')
    resultados_consulta +=str(distancias_coseno)
    resultados_consulta +=('\n\n')
    return resultados_consulta
    

**Calculamos la similitud del coseno entre todos los mensajes seleccionados del conjunto de test, con todos el conjunto de train(le pasamos el vector de test entero por que la propia funcion ya se encarga de seleccionar los mensajes seleccionados)**

In [13]:
cosine_similarities = calculate_cosine_similarity(test_vector_data,train_vector_data)

In [14]:
len(cosine_similarities)


60

**Calculamos la precisión de la lista con niveles de exhaustividad 3 y 10 para obtener los 3 y 10 primeros documentos más relevantes de cada consulta**

In [15]:
three_relevants,info_three_relevants = get_relevants(3,cosine_similarities)

In [16]:
print(info_three_relevants)

CONSULTAS CLASE 0
----Mensaje consulta nª 2
--Id's documentos recuperados:
[3172 7539 6909]
--Similitudes entre los documentos recuperados y la consulta:
[0.47172818 0.45275437 0.40089186]

----Mensaje consulta nª 14
--Id's documentos recuperados:
[9623 4066 6596]
--Similitudes entre los documentos recuperados y la consulta:
[0.35698807 0.33829381 0.32846258]

----Mensaje consulta nª 49
--Id's documentos recuperados:
[3621  582  192]
--Similitudes entre los documentos recuperados y la consulta:
[0.35750785 0.34445018 0.33957856]

CONSULTAS CLASE 1
----Mensaje consulta nª 9
--Id's documentos recuperados:
[10221   569  2497]
--Similitudes entre los documentos recuperados y la consulta:
[0.31945757 0.30139814 0.29933304]

----Mensaje consulta nª 17
--Id's documentos recuperados:
[2882 3933 4696]
--Similitudes entre los documentos recuperados y la consulta:
[0.30935922 0.29462783 0.2941742 ]

----Mensaje consulta nª 20
--Id's documentos recuperados:
[6056 8292 3916]
--Similitudes entre los

In [17]:
ten_relevants,info_ten_relevants = get_relevants(10,cosine_similarities)

In [18]:
print(info_ten_relevants)

CONSULTAS CLASE 0
----Mensaje consulta nª 2
--Id's documentos recuperados:
[3172 7539 6909 7201 6875 8810 7687 2398 8007 7831]
--Similitudes entre los documentos recuperados y la consulta:
[0.47172818 0.45275437 0.40089186 0.36596253 0.34743961 0.34646743
 0.34156503 0.33722664 0.31339159 0.30966177]

----Mensaje consulta nª 14
--Id's documentos recuperados:
[ 9623  4066  6596  6552  1862  4421  6432 11187   210  4295]
--Similitudes entre los documentos recuperados y la consulta:
[0.35698807 0.33829381 0.32846258 0.31964129 0.31619096 0.31270282
 0.31151958 0.30911784 0.30885736 0.30806671]

----Mensaje consulta nª 49
--Id's documentos recuperados:
[ 3621   582   192  1379  6116   980  5733 10575   931  3742]
--Similitudes entre los documentos recuperados y la consulta:
[0.35750785 0.34445018 0.33957856 0.33417007 0.33391244 0.32714757
 0.32544164 0.31802671 0.31642548 0.30809929]

CONSULTAS CLASE 1
----Mensaje consulta nª 9
--Id's documentos recuperados:
[10221   569  2497 11235   335

Como podemos ver en el siguiente ejemplo, si queremos ver los documentos recuperados para las consultas de la clase 0:

In [19]:
three_relevants[0]

[array([3172, 7539, 6909], dtype=int64),
 array([9623, 4066, 6596], dtype=int64),
 array([3621,  582,  192], dtype=int64)]

In [20]:
ten_relevants[0]

[array([3172, 7539, 6909, 7201, 6875, 8810, 7687, 2398, 8007, 7831],
       dtype=int64),
 array([ 9623,  4066,  6596,  6552,  1862,  4421,  6432, 11187,   210,
         4295], dtype=int64),
 array([ 3621,   582,   192,  1379,  6116,   980,  5733, 10575,   931,
         3742], dtype=int64)]

Vemos como para la primera consulta de la clase 0 hemos recuperado los mensajes con id [3172, 7539, 6909], para la segunda [9623, 4066, 6596] y para la tercera [3621,  582,  192].

Igual pasaría con los ten_relevants pero recuperando 10 mensajes en vez de 3

**A continuación crearemos dos funciones auxiliares:**

    - Una para obtener la precisión de aciertos de un conjunto de documentos
    - Otra para calcular la precisión media para cada clase del conjunto de datos(3 mensajes por clase)

In [21]:
##relevant_documents es un array que contiene id's de documentos
##target es la clase a la que deberían pertenecer los documentos 
def precision_consulta(target, relevant_documets):
    hits = 0
    total = 0
    for d in relevant_documets:
        #Si la clase supuesta del documento es igual a la que le pasamos por target(la real)
        #entonces es un acierto 
        if train_data.target[d] == target:
            hits+=1
            
        total+=1
        
    return hits/total

In [22]:


#Por parámetro recibe a la matriz de documentos recuperados por cada mensaje de cada clase
def print_precision_table(relevants_documents):
    
    target = 0
    ##Recorremos la matriz por clases
    for clase in relevants_documents:
        precision_media=0   
        #Ahora, por cada una de las 3 listas de documentos recuperados por cada clase calculamos su precisión media
        for consulta in clase:
            ## la siguiente funcion me calcula la precision de esa consulta
            precision = precision_consulta(target,consulta)
            precision_media += precision
            
        print("Precision media clase",target,"(",train_data.target_names[target],"):"
              ,precision_media/len(clase))
        target+=1
        
    

In [23]:
print_precision_table(three_relevants)

Precision media clase 0 ( alt.atheism ): 0.3333333333333333
Precision media clase 1 ( comp.graphics ): 0.2222222222222222
Precision media clase 2 ( comp.os.ms-windows.misc ): 0.2222222222222222
Precision media clase 3 ( comp.sys.ibm.pc.hardware ): 0.5555555555555555
Precision media clase 4 ( comp.sys.mac.hardware ): 0.0
Precision media clase 5 ( comp.windows.x ): 0.3333333333333333
Precision media clase 6 ( misc.forsale ): 0.7777777777777777
Precision media clase 7 ( rec.autos ): 0.3333333333333333
Precision media clase 8 ( rec.motorcycles ): 0.8888888888888888
Precision media clase 9 ( rec.sport.baseball ): 0.7777777777777777
Precision media clase 10 ( rec.sport.hockey ): 0.6666666666666666
Precision media clase 11 ( sci.crypt ): 0.5555555555555555
Precision media clase 12 ( sci.electronics ): 0.4444444444444444
Precision media clase 13 ( sci.med ): 0.7777777777777777
Precision media clase 14 ( sci.space ): 0.7777777777777777
Precision media clase 15 ( soc.religion.christian ): 0.7777

In [24]:
print_precision_table(ten_relevants)

Precision media clase 0 ( alt.atheism ): 0.2333333333333333
Precision media clase 1 ( comp.graphics ): 0.19999999999999998
Precision media clase 2 ( comp.os.ms-windows.misc ): 0.3
Precision media clase 3 ( comp.sys.ibm.pc.hardware ): 0.5
Precision media clase 4 ( comp.sys.mac.hardware ): 0.06666666666666667
Precision media clase 5 ( comp.windows.x ): 0.3
Precision media clase 6 ( misc.forsale ): 0.5666666666666668
Precision media clase 7 ( rec.autos ): 0.3
Precision media clase 8 ( rec.motorcycles ): 0.7333333333333333
Precision media clase 9 ( rec.sport.baseball ): 0.7666666666666666
Precision media clase 10 ( rec.sport.hockey ): 0.5666666666666668
Precision media clase 11 ( sci.crypt ): 0.3666666666666667
Precision media clase 12 ( sci.electronics ): 0.5
Precision media clase 13 ( sci.med ): 0.5333333333333333
Precision media clase 14 ( sci.space ): 0.7333333333333334
Precision media clase 15 ( soc.religion.christian ): 0.5
Precision media clase 16 ( talk.politics.guns ): 0.399999999

Vamos a añadir un par de funciones auxiliares para estudiar y responder a las preguntas:

In [25]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [26]:
#Muestra el array con los id's recuperados por cada una de las 3 consultas de la clase
#Conjunto_recuperados: lista de three_relevants o ten_relevants
#clase: clase de la que se quieren obtener los id's de los documentos recuperados

def mensajesRecuperados(clase,conjunto_recuperados):
    print("Mensajes de consulta(del conjunto test) para la clase ",test_data.target_names[clase],": ")
    print(lista_mensajes[clase])
    print("-Mensajes recuperados: ")
    i=0
    for d in conjunto_recuperados[clase]:
        print("--Consulta",lista_mensajes[clase][i],":\n")
        print("Is's docs recuperados:")
        print(d)
        print("Targets docs recuperados:")
        realTargets(clase,conjunto_recuperados[clase][i])
        print("\n")
        i+=1
    
    

In [27]:
#Muestra la clase a la que realmente pertenecen los documentos recuperados

def realTargets(clase,conjunto_recuperados):
    print(train_data.target[conjunto_recuperados])

## PREGUNTAS:
Contesta a lo siguiente:

         -¿Hay muchas diferencias entre los valores de precisión medios para las distintas clases del conjunto de datos? ¿A qué crees que se deben?
         
         -Identifica la clase que haya tenido peores resultados de precisión y para alguna de sus consultas muestra alguno de los mensajes que recuperó erróneamente en las primeras X posiciones.¿Con qué clases se ha confundido más dicha consulta?¿A qué crees que se deben los malos resultados?


Miraremos la tabla de precisión de 10 documentos recuperados para poder estudiar mejor los valores, y calcular un valor más preciso.

Si miramos la precisión de las clases,si que podemos ver bastante diferencias.
Podemos ver que hay clases con un valor de precisión media bastante alto como por ejemplo sci.space  (0.7333) ,rec.sport.baseball (0.7666),rec.motorcycles(0.7333) entre algunos otros. Los documentos recuperados suelen ser más relevantes en esta clase debido a que son categorías más "cerradas", es decir, que tiene palabras muy significativas que suelen aparecer en la clase baseball.Por ejemplo en estos mensajes aparecen palabras como 'baseball','runner','squares','hitters','hit', 'defender','pitch'...

**Mensajes consulta de baseball**

Vamos a ver los mensajes recuperados y sus clases reales:

In [28]:
mensajesRecuperados(9,ten_relevants)

Mensajes de consulta(del conjunto test) para la clase  rec.sport.baseball : 
[36, 53, 69]
-Mensajes recuperados: 
--Consulta 36 :

Is's docs recuperados:
[ 7974  7953 10350   540  5879 11263  9952  8641  4354   114]
Targets docs recuperados:
[9 9 9 9 9 9 9 9 9 9]


--Consulta 53 :

Is's docs recuperados:
[6098 9605 4560 7559 1991 7088 8659 7810 1651  586]
Targets docs recuperados:
[10  9  9  9  9  9  9 10  9  9]


--Consulta 69 :

Is's docs recuperados:
[ 6492  3075  1967  7026  8424  8635  6884  4123  8537 10579]
Targets docs recuperados:
[ 0  9  9  9  2 19  0  9 10  9]




Podemos ver que el mensaje 36 ha recuperado los 10 documentos de la clase correcta, el mensaje 53 lo ha hecho con bastante preición, mientras que el mensaje 69 ha sido un poco menos preciso. Analicemos un par de los que han recuperado erroneamente con esta última consulta:

In [29]:
#Mensaje 69(consulta)
write_terms(feature_names,None,test_vector_data,69)

Mensaje 69 vectorizado: ['action' 'actually' 'amazement' 'article' 'association' 'balk' 'baseball'
 'batter' 'believe' 'bunt' 'consider' 'does' 'ed' 'following' 'game'
 'games' 'happened' 'happening' 'honestly' 'years' 'lines' 'marginally'
 'matter' 'missed' 'organization' 'pair' 'piece' 'played' 'posting'
 'prevents' 'puts' 'quite' 'realistic' 'remove' 'responsibility'
 'ridiculous' 'rule' 'run' 'runner' 'runners' 'safe' 'scenario' 'seeing'
 'speed' 'squares' 'squeeze' 'stay' 'starts' 'stops' 'subject'
 'suggestions' 'suicide' 'takes' 'throw' 'umpire' 'umpiring' 'worst'
 'writes'] 



Si miramos los mensajes recuperados 8537 (clase hockey) y 6492 (clase atheism) como haremos a continuación, vemos que han sido clasificados de la clase hockey por palabras que aparecen en la consulta, como por ejemplo "article","games","lines","posting","writes"...

Algunas es normal que aparezcan ya que al fin y al cabo estas hablando sobre deportes, y muchas palabras son usadas en la mayoria de los deportes, como "games" y "lines" por ejemplo. Así hemos recuperado este documento de la clse hockey, ya que tiene varias palabras en común con este mensaje.

In [30]:
#Mensajes recuperados 8537 y 6492
write_terms(feature_names,None,train_vector_data,8537)
print("\n")
print(train_data.data[8537])

Mensaje 8537 vectorizado: ['article' 'championships' 'games' 'lines' 'organization' 'posting'
 'results' 'shown' 'subject' 'thanks' 'world' 'writes'] 



From: ee152fcs@sdcc15.ucsd.edu (Bjorn Karlsson)
Subject: Re: WC 93: Results, April 20
Organization: University of California, San Diego
Lines: 12
Nntp-Posting-Host: sdcc15.ucsd.edu

In article <1993Apr21.073134.5117@ericsson.se> etxonss@ufsa.ericsson.se (Staffan Axelsson) writes:
>
> 1993 World Championships in Germany:
> ====================================

Is there any games being shown here in the US from the WC???

Thanks

mc





Si miramos ahora el mensaje 6492, podemos ver que lo ha clasificado en la clase hockey por la coincidencia de ciertas palabras como article, lines,writes,posting... que son palabras que aparecen en muchos documentos, aunque ninguna relevante para relacionarlo realmente con el baseball.

In [31]:
write_terms(feature_names,None,train_vector_data,6492)
print("\n")
print(train_data.data[6492])

Mensaje 6492 vectorizado: ['accept' 'article' 'atheism' 'computer' 'department' 'does' 'inbreeding'
 'lines' 'mean' 'orbit' 'organization' 'posting' 'quiz' 'religions'
 'science' 'starts' 'subject' 'trashing' 'usenet' 'writes'] 



From: kellyb@ccsua.ctstateu.edu
Subject: Re: Bible Quiz
Lines: 12
Nntp-Posting-Host: ccsua.ctstateu.edu
Organization: Yale University, Department of Computer Science, New Haven, CT

In article <kmr4.1563.734805744@po.CWRU.edu>, kmr4@po.CWRU.edu (Keith M. Ryan) writes:
> In article <1qgbmt$c4f@usenet.INS.CWRU.Edu> cr866@cleveland.Freenet.Edu (Frank D. Kirschner) writes:
> 
>> ---
> 
>    Only when the Sun starts to orbit the Earth will I accept the Bible. 
>         
> 
     Since when does atheism mean trashing other religions?There must be a God
     of inbreeding to which you are his only son.

                                                  Pope John Paul



**Mensajes consulta de sci.space**

Si miramos las palabras de los mensajes seleccionados del conjunto de test de la clase space, vemos que estos mensajes también tienen palabras que dificilmente aparecerán en otro tipo de documentos mientras que en esta clase abundan, como por ejemplo: electrostatic,propulsion,spacecraft,ultraviolet,heliocentric...
Tienen en general un léxico muy científico que hace la consulta más facil.

In [32]:
mensajesRecuperados(14,ten_relevants)

Mensajes de consulta(del conjunto test) para la clase  sci.space : 
[23, 34, 46]
-Mensajes recuperados: 
--Consulta 23 :

Is's docs recuperados:
[8508 4854 1264 7198 9664 5311 7944 2209 4530 7300]
Targets docs recuperados:
[14 11 14  1 14  9 11 14 14 14]


--Consulta 34 :

Is's docs recuperados:
[ 4706  5071  4767  4856  5066  9095  3031 10873  6220  2762]
Targets docs recuperados:
[14 14 14 14 14 14 14 14 14 14]


--Consulta 46 :

Is's docs recuperados:
[10165  8934  4412  9073  8039  9611  4141  1027  8613  8047]
Targets docs recuperados:
[14 14 12 12 14 14  8 14 14  8]




Para la primera consulta(mensaje 23), vemos que lo ha confundido con la clase 9 (baseball), vamos a analizar este caso:

In [33]:
write_terms(feature_names,None,test_vector_data,23)

Mensaje 23 vectorizado: ['access' 'array' 'arrays' 'article' 'bother' 'communications'
 'contributed' 'days' 'design' 'designed' 'detachable' 'dictated'
 'express' 'extendable' 'flapping' 'indirectly' 'lines' 'mission' 'online'
 'organization' 'pinch' 'posting' 'problems' 'release' 'remove'
 'retractable' 'retrievable' 'scheduled' 'second' 'servicing' 'shuttle'
 'solar' 'space' 'subject' 'thing' 'walk' 'writes' 'zoo'] 



In [34]:
write_terms(feature_names,None,train_vector_data,5311)
print("\n")
print(train_data.data[5311])

Mensaje 5311 vectorizado: ['access' 'communications' 'distribution' 'does' 'express' 'lines' 'long'
 'online' 'organization' 'posting' 'random' 'scoop' 'subject'] 



From: random@access.digex.com (Random)
Subject: Scott Erickson
Organization: Express Access Online Communications, Greenbelt, MD USA
Lines: 3
Distribution: usa
NNTP-Posting-Host: access.digex.net

Does anyone have the scoop on Scot Erickson?  How long is he going to be
out for?




Encuentra que aparecen las palabras "access","communications","express","lines","online","organization","posting" y "subject", que aunque tengan unas cuantas apariciones de palabras en común con la consulta, no guardan ninguna relación con el tema del espacio. 

La consulta 34 ha recuperado correctamente 10 documentos por su abundancia de palabras relevantes,ya que tiene un amplio repertorío de palabras relacionadas con la ciencia y el espacio.

In [35]:
write_terms(feature_names,None,test_vector_data,34)

Mensaje 34 vectorizado: ['ac' 'acceptable' 'accomplishments' 'acquisition' 'action' 'activity'
 'activities' 'analysis' 'anomaly' 'antenna' 'antennas' 'approximately'
 'areas' 'attitude' 'aweto' 'change' 'changes' 'charge' 'cycle' 'closed'
 'coded' 'command' 'commanded' 'commands' 'completion' 'conducted'
 'configuration' 'consecutive' 'consistent' 'contacts' 'continuous'
 'control' 'cruise' 'current' 'deep' 'degree' 'degrees' 'delayed'
 'delivery' 'demonstrated' 'design' 'detector' 'developed' 'development'
 'differenced' 'director' 'discussed' 'distance' 'downlink' 'dual'
 'electrostatic' 'engineers' 'execute' 'executed' 'execution' 'exhibited'
 'extreme' 'follows' 'forwarded' 'general' 'goldstone' 'ground' 'group'
 'hardware' 'heliocentric' 'hour' 'imbalance' 'included' 'increased'
 'indicate' 'indicated' 'indicates' 'initial' 'initiated' 'initiation'
 'institutional' 'instrument' 'instruments' 'interfaced' 'internal'
 'keywords' 'km' 'laboratory' 'lagging' 'launch' 'leading' 'lines

En conclusión, estas clases tienen más precisión ya que tienen palabras frecuentan en estos tipos de clases, y que las hace destacar a la hora de ser recuperados.

Si ahora miramos aquellos que tienen unos valores de precision  bastante bajos, como por ejemplo: comp.graphics: 0.199,comp.sys.mac.hardware: 0.0666, comp.windows.x: 0.3,talk.politics.misc: 0.233 ,talk.religion.misc: 0.1666; vemos que son categorías que se pueden confundir bastante entre sí:

Por ejemplo, todas las categorías que tienen que ver con informática se van a confundir bastante entre sí ya que en sus documentos existe un léxico parecido y al estar usando bolsa de palabras con frecuencias nos estamos basando en el número de veces que aparecen las palabras de la consulta en el documento(con el valor tfidf mejoraría la clasificación de este nos ayuda a dar un mayor peso a aquellas que no suelen aparecer en otros documentos, así solucionariamos este problema al tener un léxico muy parecido aunque trate diferentes categorías).

También hay que tener en cuenta que estos valores de precision podrían alterarse a mejor o a peor, dependiendo de los mensajes que hayamos seleccionado como consultas, ya que si elegimos un mensaje que contenga palabras "clave" con más relevancia para clasificarlo correctamente obtendríamos una mejora de precisión, mientras que si estamos intentando recuperar documentos en base a una consulta que no tiene palabras muy relevantes o que sean muy generales y utilizadas en otras clases de documentos, entonces tendriamos una precisión más baja. En nuestro caso ha sido selección aleatoría de los documentos, pero lo acertado sería encontrar un mensaje que pueda tener un léxico más significativo para la clase.

**Peor precision en consultas:**

Si miramos la tabla de precisión (en la que recuperamos 10) vemos que la clase que tiene peores resultados es la número 4 (comp.sys.mac.hardware).Tiene una precisión media de 0.066666.

In [36]:
mensajesRecuperados(4,ten_relevants)

Mensajes de consulta(del conjunto test) para la clase  comp.sys.mac.hardware : 
[16, 32, 37]
-Mensajes recuperados: 
--Consulta 16 :

Is's docs recuperados:
[10591  4055  8216  5090  3405  7264  4267  6221  8156  8945]
Targets docs recuperados:
[2 2 2 2 2 2 2 4 2 2]


--Consulta 32 :

Is's docs recuperados:
[4453 6680 6452  159 4992 9692 4278 3502 2331  969]
Targets docs recuperados:
[ 5  6 12 15  5  6  6  5 12  4]


--Consulta 37 :

Is's docs recuperados:
[10966  2695  4696  5571  2331  8484  2433  4474  5962  9294]
Targets docs recuperados:
[18  5 18  6 12  6  1  6  6  7]




Podemos ver que el mensaje 16 del conjunto de test nos recupera casi todo mensajes de la clase 2 (comp.os.ms-windows.misc), y si vemos el mensaje 16 y algunos de los recuperados podemos ver que en estos aparecen varias palabras presentes en el mensaje 16.

También lo ha confundido con la clase 5 (comp.windows.x) y la 6 (sci.electronics), que como habíamos comentado en la anterior pregunta, son clases con un léxico muy parecido).

Vamos a ver algún mensaje de los recuperados erroneamente:

In [37]:
#mensaje 16
write_terms(feature_names,None,test_vector_data,16)
print(test_data.data[16])

Mensaje 16 vectorizado: ['boxes' 'college' 'couple' 'different' 'disk' 'documented' 'error' 'file'
 'fills' 'fonts' 'got' 'gradient' 'happen' 'happens' 'help' 'ideas'
 'letters' 'lines' 'organization' 'parts' 'print' 'printing' 'problem'
 'quark' 'subject' 'text' 'thanks' 'things' 'trying' 'uneffected'] 

From: jacobs@cerritos.edu
Subject: Problem printing Quark on a SWII
Organization: Cerritos College, Norwalk CA
Lines: 15

Iv'e got a problem printing with a StyleWriterII. I am printing from a IIvx
with 20 megs ram. I am trying to print a Quark file that has 2 fonts a couple
of boxes and 3 gradient fills. 

Two things happen: I get a " Disk is full" error, that I can't find documented,
I also have parts of letters that are over one of the gradient fills get cut
off. This only happens to the text over the fill. Text adjecent in a different
box is uneffected.

Any ideas?

Thanks for the help...-- 
_______________________________________________________________________________
Karl Jacob

In [38]:
#clase 2(comp.os.ms-windows.misc)
write_terms(feature_names,None,train_vector_data,10591)
print("\n")
print(train_data.data[10591])

Mensaje 10591 vectorizado: ['advance' 'answer' 'bought' 'correct' 'disable' 'driver' 'end' 'envelops'
 'feeds' 'got' 'groups' 'history' 'landscape' 'lines' 'macro' 'modify'
 'narrow' 'organization' 'orientation' 'portrait' 'posting' 'present'
 'print' 'printing' 'problem' 'reverse' 'semester' 'subject' 'summarize'
 'thanks' 'wants' 'windows'] 



From: rcs1@crux3.cit.cornell.edu (R Craig Stevenson)
Subject: W4W: printing envelops on DJ550C (not 500C!)
Nntp-Posting-Host: crux3.cit.cornell.edu
Organization: Cornell University
Lines: 19

I've got a probelm with printing envelops on my deskjet 550C from Word
for Windows.

History: when I had a Deskjet 500, I had to modify the macro so that
it would disable reverse printing since there seemed to be a bug that
interferred with printing envelops.  That bug is still present, however...

Now: the problem I have is that my DeskJet 550 print driver (came with
the printer I bought in December) still wants to print the envelops in
PORTRAIT mode.  H

En el mensaje aparecen las siguientes palabras en común con el mensaje consulta: "got","lines","organization","print", "printing"... seguimos teniendo palabras muy generales aunque esten en común con el mensaje 16.

Con la consulta 32 recuperamos los documentos 6680 y 4474 perteneciente a la clase sci.electronics:

In [39]:
#mensaje 32
write_terms(feature_names,None,test_vector_data,32)


Mensaje 32 vectorizado: ['disabling' 'disk' 'duty' 'eject' 'exists' 'information' 'know' 'lines'
 'macs' 'organization' 'program' 'programmer' 'spitting' 'stop' 'subject'
 'technology' 'thanks' 'told' 'trying'] 



In [40]:
##clase 6(sci.electronics)
write_terms(feature_names,None,train_vector_data,6680)
print("\n")
print(train_data.data[6680])

Mensaje 6680 vectorizado: ['computer' 'distribution' 'institute' 'internet' 'lines' 'organization'
 'subject' 'technology'] 



From: gt4661a@prism.gatech.EDU (gt4661a gt4661a PAOLO,MARC ANTHONY)
Subject: Computer For Sale
Distribution: atl
Organization: Georgia Institute of Technology
Lines: 5

-- 
PAOLO,MARC ANTHONY
Georgia Institute of Technology, Atlanta Georgia, 30332
uucp:     ...!{allegra,amd,hplabs,ut-ngp}!gatech!prism!gt4661a
Internet: gt4661a@prism.gatech.edu



Aparecen las palabras "lines","organization","subject","technology".
Sigue pasando lo mismo con todos los mensajes, las palabras de este mensaje aparecen en la consulta pero son demasiados generales para diferenciar la clase a la que pertenecen.

In [41]:
##clase 6(sci.electronics)
write_terms(feature_names,None,train_vector_data,4474)

Mensaje 4474 vectorizado: ['article' 'audio' 'distribution' 'lines' 'misc' 'organization' 'posting'
 'subject'] 



In [42]:
print(train_data.data[4474])

From: hungjenc@phakt.usc.edu (Hung-Jen Chen)
Subject: Forsale: Sony D-22 diskman
Article-I.D.: phakt.1pqm89INNja1
Distribution: na
Organization: University of Southern California, Los Angeles, CA
Lines: 4
NNTP-Posting-Host: phakt.usc.edu


Newsgroups: rec.audio,misc.forsale
Distribution: na
Subject: Forsale: Sony D-22 diskman



Como podemos ver pasa lo que habíamos supuesto, que el léxico de los mensajes de la clase 2, de la clase 6 y de la clase 4 es bastante parecido.

## Apartado b)

**Ahora realizaremos lo mismo pero en vez de con las frecuencias las palabras, lo haremos con una bolsa de palabras con TF/IDF**

In [43]:
# Calculamos el valor TF-IDF para nuestros dos vectores
tfidfer = TfidfTransformer()
train_preprocessed = tfidfer.fit_transform(train_vector_data)
test_preprocessed=tfidfer.transform(test_vector_data)

**Calculamos la distancia del coseno entre los 60 mensajes del conjunto test (3 mensajes por cada una de las 20 clases) y el cojunto de train.**

In [44]:
cosine_similarities_tdidf = calculate_cosine_similarity(test_preprocessed,train_preprocessed)

**Obtenemos los 3 y los 10 documentos más relevantes de cada consulta:**

In [45]:
three_relevants_tfidf,info_three_relevants_tfidf = get_relevants(3,cosine_similarities_tdidf)

In [46]:
print(info_three_relevants_tfidf)

CONSULTAS CLASE 0
----Mensaje consulta nª 2
--Id's documentos recuperados:
[7539 3172 6875]
--Similitudes entre los documentos recuperados y la consulta:
[0.41469713 0.38118576 0.32666042]

----Mensaje consulta nª 14
--Id's documentos recuperados:
[ 210 8795 9623]
--Similitudes entre los documentos recuperados y la consulta:
[0.18181959 0.16976696 0.16739938]

----Mensaje consulta nª 49
--Id's documentos recuperados:
[ 192 3621 6116]
--Similitudes entre los documentos recuperados y la consulta:
[0.30162446 0.29884185 0.27010764]

CONSULTAS CLASE 1
----Mensaje consulta nª 9
--Id's documentos recuperados:
[ 569 2497 3106]
--Similitudes entre los documentos recuperados y la consulta:
[0.2584884  0.21679884 0.21237681]

----Mensaje consulta nª 17
--Id's documentos recuperados:
[2172 6387 7608]
--Similitudes entre los documentos recuperados y la consulta:
[0.21755868 0.21277376 0.1952026 ]

----Mensaje consulta nª 20
--Id's documentos recuperados:
[5018 8936 6056]
--Similitudes entre los do

In [47]:
ten_relevants_tfidf,info_ten_relevants_tfidf = get_relevants(10,cosine_similarities_tdidf)

In [48]:
print(info_ten_relevants_tfidf)

CONSULTAS CLASE 0
----Mensaje consulta nª 2
--Id's documentos recuperados:
[ 7539  3172  6875  7201  6909  8007 10749  2095  8810  2398]
--Similitudes entre los documentos recuperados y la consulta:
[0.41469713 0.38118576 0.32666042 0.29853306 0.29111836 0.26190998
 0.25422292 0.2370712  0.2280488  0.22685931]

----Mensaje consulta nª 14
--Id's documentos recuperados:
[ 210 8795 9623 1749 6013 6552 2840  221 5200 7480]
--Similitudes entre los documentos recuperados y la consulta:
[0.18181959 0.16976696 0.16739938 0.16649032 0.15579455 0.15504036
 0.15349439 0.15041995 0.14910271 0.14754671]

----Mensaje consulta nª 49
--Id's documentos recuperados:
[ 192 3621 6116  582 5733 3742 1497  931  980 5144]
--Similitudes entre los documentos recuperados y la consulta:
[0.30162446 0.29884185 0.27010764 0.25812669 0.25595316 0.25022165
 0.23934987 0.23285171 0.23268068 0.22360311]

CONSULTAS CLASE 1
----Mensaje consulta nª 9
--Id's documentos recuperados:
[  569  2497  3106  1083   709 10747 102

**Calculamos los valores de precisión media para cada clase del conjunto de datos**

Precisión para un nivel de exhaustividad 3:

In [49]:
##tdifd
print_precision_table(three_relevants_tfidf)

Precision media clase 0 ( alt.atheism ): 0.2222222222222222
Precision media clase 1 ( comp.graphics ): 0.1111111111111111
Precision media clase 2 ( comp.os.ms-windows.misc ): 0.1111111111111111
Precision media clase 3 ( comp.sys.ibm.pc.hardware ): 0.6666666666666666
Precision media clase 4 ( comp.sys.mac.hardware ): 0.3333333333333333
Precision media clase 5 ( comp.windows.x ): 0.3333333333333333
Precision media clase 6 ( misc.forsale ): 0.4444444444444444
Precision media clase 7 ( rec.autos ): 0.4444444444444444
Precision media clase 8 ( rec.motorcycles ): 0.7777777777777778
Precision media clase 9 ( rec.sport.baseball ): 1.0
Precision media clase 10 ( rec.sport.hockey ): 1.0
Precision media clase 11 ( sci.crypt ): 0.5555555555555555
Precision media clase 12 ( sci.electronics ): 0.4444444444444444
Precision media clase 13 ( sci.med ): 0.7777777777777777
Precision media clase 14 ( sci.space ): 1.0
Precision media clase 15 ( soc.religion.christian ): 0.7777777777777777
Precision media c

Precisión para un nivel de exhaustividad 10:

In [50]:
#tfid
print_precision_table(ten_relevants_tfidf)

Precision media clase 0 ( alt.atheism ): 0.26666666666666666
Precision media clase 1 ( comp.graphics ): 0.3
Precision media clase 2 ( comp.os.ms-windows.misc ): 0.26666666666666666
Precision media clase 3 ( comp.sys.ibm.pc.hardware ): 0.5666666666666668
Precision media clase 4 ( comp.sys.mac.hardware ): 0.20000000000000004
Precision media clase 5 ( comp.windows.x ): 0.4333333333333333
Precision media clase 6 ( misc.forsale ): 0.39999999999999997
Precision media clase 7 ( rec.autos ): 0.5
Precision media clase 8 ( rec.motorcycles ): 0.7333333333333333
Precision media clase 9 ( rec.sport.baseball ): 0.8666666666666667
Precision media clase 10 ( rec.sport.hockey ): 1.0
Precision media clase 11 ( sci.crypt ): 0.3666666666666667
Precision media clase 12 ( sci.electronics ): 0.6
Precision media clase 13 ( sci.med ): 0.6666666666666666
Precision media clase 14 ( sci.space ): 0.8666666666666667
Precision media clase 15 ( soc.religion.christian ): 0.6
Precision media clase 16 ( talk.politics.gu

In [129]:
#Con frecuencias,para comparar con la tabla de 10 relevantes tfidf anterior
print_precision_table(ten_relevants) 

Precision media clase 0 ( alt.atheism ): 0.2333333333333333
Precision media clase 1 ( comp.graphics ): 0.19999999999999998
Precision media clase 2 ( comp.os.ms-windows.misc ): 0.3
Precision media clase 3 ( comp.sys.ibm.pc.hardware ): 0.5
Precision media clase 4 ( comp.sys.mac.hardware ): 0.06666666666666667
Precision media clase 5 ( comp.windows.x ): 0.3
Precision media clase 6 ( misc.forsale ): 0.5666666666666668
Precision media clase 7 ( rec.autos ): 0.3
Precision media clase 8 ( rec.motorcycles ): 0.7333333333333333
Precision media clase 9 ( rec.sport.baseball ): 0.7666666666666666
Precision media clase 10 ( rec.sport.hockey ): 0.5666666666666668
Precision media clase 11 ( sci.crypt ): 0.3666666666666667
Precision media clase 12 ( sci.electronics ): 0.5
Precision media clase 13 ( sci.med ): 0.5333333333333333
Precision media clase 14 ( sci.space ): 0.7333333333333334
Precision media clase 15 ( soc.religion.christian ): 0.5
Precision media clase 16 ( talk.politics.guns ): 0.399999999


## PREGUNTAS:
A continuación contesta a lo siguiente.

    •¿Han cambiado los valores de precisión media para las clases del conjunto de datos?¿Qué clases han mejorado? ¿Cuáles han empeorado?
    
    •Encuentra una consulta donde el uso de la ponderación TF-IDF haya sido efectivo y haya mejorado los resultados. Explica por qué ha sido efectivo.

Al haber aplicado la ponderación tf/idf, estamos quitando peso a aquellas que no son muy relevantes ya que aparecen en muchos documentos; y dando más peso a aquellas que no suelen aparecer en otros documentos.
Así, como podemos observar, con esta bolsa de palabras aumenta la precisión de prácticamente todas las consultas. En algunas el aumento de la precisión es muy pequeño, mientras que tenemos otros cambios más significativos.

Veamos las que no han mejorado:

    -comp.os.ms-windows.misc : 0.3->0.266
    -misc.forsale: 0.56->0.39
    -sci.crypt : mantiene misma precisión, 0.36
    -rec.motorcycles: mantiene misma precisión, 0,733
    
Todas las demás han mejorado,y las más notables han sido las siguientes:

    -talk.politics.mideast (17): 0.266->0,63
    -rec.sport.hockey (10): 0.56->1.0
    -comp.sys.mac.hardware (4): 0.066->0.20

**Para la clase 4, que había sido la que peor precisión tenia antes, ahora obtenemos los siguientes documentos:**

In [131]:
mensajesRecuperados(4,ten_relevants_tfidf)

Mensajes de consulta(del conjunto test) para la clase  comp.sys.mac.hardware : 
[16, 32, 37]
-Mensajes recuperados: 
--Consulta 16 :

Is's docs recuperados:
[ 4055  1550  5090  8216 10591  5319  8843  7264  9891  1693]
Targets docs recuperados:
[2 6 2 2 2 6 6 2 6 2]


--Consulta 32 :

Is's docs recuperados:
[ 159 6680 3140 8125 6452 4992 2700 5955 9692 5786]
Targets docs recuperados:
[15  6  4  5 12  5  6  4  6  3]


--Consulta 37 :

Is's docs recuperados:
[6447 4609 5962 7543 2695 8344  457 4940 9294 6193]
Targets docs recuperados:
[ 4  4  6  3  5  4 11  2  7  4]




Antes teniamos:

In [53]:
mensajesRecuperados(4,ten_relevants)

Mensajes de consulta(del conjunto test) para la clase  comp.sys.mac.hardware : 
[16, 32, 37]
-Mensajes recuperados: 
--Consulta 16 :

Is's docs recuperados:
[10591  4055  8216  5090  3405  7264  4267  6221  8156  8945]
Targets docs recuperados:
[2 2 2 2 2 2 2 4 2 2]


--Consulta 32 :

Is's docs recuperados:
[4453 6680 6452  159 4992 9692 4278 3502 2331  969]
Targets docs recuperados:
[ 5  6 12 15  5  6  6  5 12  4]


--Consulta 37 :

Is's docs recuperados:
[10966  2695  4696  5571  2331  8484  2433  4474  5962  9294]
Targets docs recuperados:
[18  5 18  6 12  6  1  6  6  7]




Vemos como por ejemplo, usando de consulta el mensaje 16 , recuperamos al documento 10591 tanto con bolsa de palabra con frecuencias como con bolsa de palabras con valores tf/idf.La diferencia es que la similitud del coseno es mayor cuando usamos frecuencias que cuando usamos tf/idf(que hará que la similitud disminuya):

In [55]:
write_terms(feature_names,None,test_vector_data,16)

Mensaje 16 vectorizado: ['boxes' 'college' 'couple' 'different' 'disk' 'documented' 'error' 'file'
 'fills' 'fonts' 'got' 'gradient' 'happen' 'happens' 'help' 'ideas'
 'letters' 'lines' 'organization' 'parts' 'print' 'printing' 'problem'
 'quark' 'subject' 'text' 'thanks' 'things' 'trying' 'uneffected'] 



In [57]:
write_terms(feature_names,None,train_vector_data,10591)
print("\n")
#Descomentar linea de abajo para ver el mensaje
#print(train_data.data[10591]) 

Mensaje 10591 vectorizado: ['advance' 'answer' 'bought' 'correct' 'disable' 'driver' 'end' 'envelops'
 'feeds' 'got' 'groups' 'history' 'landscape' 'lines' 'macro' 'modify'
 'narrow' 'organization' 'orientation' 'portrait' 'posting' 'present'
 'print' 'printing' 'problem' 'reverse' 'semester' 'subject' 'summarize'
 'thanks' 'wants' 'windows'] 





Al usar frecuencias obtenemos que aparecen :got,lines,organization,print,printing,problem y subject.Son unas cuantas palabras por lo que tiene una similitud alta.

Al usar tf/idf , se rebajará el peso de todas las palabras, ya que son bastante frecuentes en el resto de documentos, son palabras muy genéricas.

Así, en el primer caso el documento esta en primera posición de los 10 recuperados, mientras que en el segundo caso está en quinta posición. Aún asi, este caso sigue estanto equivocado.

Si miramos el documento 3140( y pertence a la clase 4) que se ha recuperado al usar la consulta con el mensaje 32 y que con frecuencias no se habia recuperado:

In [60]:
write_terms(feature_names,None,test_vector_data,32)

Mensaje 32 vectorizado: ['disabling' 'disk' 'duty' 'eject' 'exists' 'information' 'know' 'lines'
 'macs' 'organization' 'program' 'programmer' 'spitting' 'stop' 'subject'
 'technology' 'thanks' 'told' 'trying'] 



In [59]:
write_terms(feature_names,None,train_vector_data,3140)
print("\n")
#Descomentar linea de abajo para ver el mensaje
#print(train_data.data[3140]) 

Mensaje 3140 vectorizado: ['buy' 'lines' 'macs' 'organization' 'posting' 'subject' 'suck'] 





Con frecuencias no habiamos obtenido este mensaje ya que aunque tenga palabras como lines,organization,subject,macs; son pocas apariciones y recupera otros documentos erroneamente basandose en que haya más apariciones de las palabras del mensaje consulta(como hemos estado viendo). Pero en el momento que lo hacemos con tf/idf, le quita bastante peso a todas las palabras que hemos mencionado al inicio del parrafo(aparecen en muchos documentos) menos a la palabra macs, la cual tendrá un gran peso, ya que "macs" es una palabra muy significativa en esta clase(comp.sys.mac.hardware) que se trata de mensajes sobre el hardware de mac.

**Para la clase 14 (sci.space), hemos obtenido una mejora(de 0,73 a 0,86):**

In [132]:
mensajesRecuperados(14,ten_relevants_tfidf)

Mensajes de consulta(del conjunto test) para la clase  sci.space : 
[23, 34, 46]
-Mensajes recuperados: 
--Consulta 23 :

Is's docs recuperados:
[8508 7506 1264 7198 4854  322 1769 5311 5880 8694]
Targets docs recuperados:
[14 14 14  1 11 14 14  9 14 14]


--Consulta 34 :

Is's docs recuperados:
[ 4706  5071  4767  5066  9095  4856  3031 10873  2762  6220]
Targets docs recuperados:
[14 14 14 14 14 14 14 14 14 14]


--Consulta 46 :

Is's docs recuperados:
[10165  8934  8508  3564  5880  4767  7848  4412 11116  9758]
Targets docs recuperados:
[14 14 14 14 14 14 14 12 14 14]




In [66]:
mensajesRecuperados(14,ten_relevants)

Mensajes de consulta(del conjunto test) para la clase  sci.space : 
[23, 34, 46]
-Mensajes recuperados: 
--Consulta 23 :

Is's docs recuperados:
[8508 4854 1264 7198 9664 5311 7944 2209 4530 7300]
Targets docs recuperados:
[14 11 14  1 14  9 11 14 14 14]


--Consulta 34 :

Is's docs recuperados:
[ 4706  5071  4767  4856  5066  9095  3031 10873  6220  2762]
Targets docs recuperados:
[14 14 14 14 14 14 14 14 14 14]


--Consulta 46 :

Is's docs recuperados:
[10165  8934  4412  9073  8039  9611  4141  1027  8613  8047]
Targets docs recuperados:
[14 14 12 12 14 14  8 14 14  8]




Por ejemplo. ahora el 4854 tiene un valor de similitud menor que con frecuencias por lo mismo que con los demás casos, quita relevancia a las palabras que aparecen en demasiados documentos.

In [67]:
write_terms(feature_names,None,test_vector_data,23)

Mensaje 23 vectorizado: ['access' 'array' 'arrays' 'article' 'bother' 'communications'
 'contributed' 'days' 'design' 'designed' 'detachable' 'dictated'
 'express' 'extendable' 'flapping' 'indirectly' 'lines' 'mission' 'online'
 'organization' 'pinch' 'posting' 'problems' 'release' 'remove'
 'retractable' 'retrievable' 'scheduled' 'second' 'servicing' 'shuttle'
 'solar' 'space' 'subject' 'thing' 'walk' 'writes' 'zoo'] 



In [69]:
write_terms(feature_names,None,train_vector_data,4854)
print("\n")
#Descomentar linea de abajo para ver el mensaje
#print(train_data.data[4854]) 

Mensaje 4854 vectorizado: ['access' 'code' 'communications' 'design' 'difficult' 'distribution'
 'express' 'gibberish' 'gives' 'lines' 'matter' 'monitor' 'online'
 'organization' 'posting' 'screen' 'shouldn' 'similar' 'subject' 'tapped'] 





**Por último veremos las consultas de la clase hockey, la cual ha recuperado ahora 10 de 10 elementos relevantes:**

In [ ]:
mensajesRecuperados(10,ten_relevants_tfidf)

In [ ]:
mensajesRecuperados(10,ten_relevants)

Usando de consulta el mensaje 56, con frecuencias obtenemos como segundo documento más relevante el 721( de la clase talk.religion.misc,lo cuál no tiene mucho que ver con el hockey),mientras que en los recuperados con tf/idf no aparece este.

In [73]:
write_terms(feature_names,None,test_vector_data,56)

Mensaje 56 vectorizado: ['beat' 'caps' 'come' 'cup' 'isles' 'lines' 'make' 'organization'
 'playoffs' 'posting' 'prove' 'ratio' 'said' 'subject' 'watch' 'winning'
 'wouldn' 'wrong'] 



In [74]:
write_terms(feature_names,None,train_vector_data,721)
print("\n")
#Descomentar linea de abajo para ver el mensaje
#print(train_data.data[721])

Mensaje 721 vectorizado: ['blessed' 'hear' 'yea' 'lines' 'organization' 'posting' 'promise' 'said'
 'software' 'subject'] 





Aparecen las mismas palabras de siempre que abundan en los malos resultados con bolsa de palabras con frecuencias:'lines' 'organization' 'posting' ,'said'
 y 'subject'. 
 Con tf/idf no aparece ya que la similitud disminuye al quitarle todo el peso que le daban esas palabras tan genéricas.
 
Sin embargo recuperamos correctamente otros documentos que no habiamos conseguido antes con frecuencias como por ejemplo:

In [81]:
write_terms(feature_names,None,train_vector_data,679)
print("\n")
#Descomentar linea de abajo para ver el mensaje
#print(train_data.data[679])

Mensaje 679 vectorizado: ['advance' 'article' 'beat' 'distribution' 'ee' 'excited' 'islanders'
 'isles' 'keeping' 'lines' 'mail' 'organization' 'overtime' 'rangers'
 'request' 'subject' 'thanks' 'writes'] 





El documento 679 aunque tiene alguna palabra en común(que no le da un peso significativo como "lines" etc.), lo que le hace estar bastante relacionado con la consulta son la palabra **islander e isles** que se refieren a un equipo de hockey.Por lo tanto, nos recuperará mensajes sobre hockey que comenten algo sobre el equipo.

Lo mismo pasa con el documento 10120:

In [84]:
write_terms(feature_names,None,train_vector_data,10120)
print("\n")
#Descomentar linea de abajo para ver el mensaje
#print(train_data.data[10120])

Mensaje 10120 vectorizado: ['1st' '2nd' '3rd' 'admit' 'afraid' 'agree' 'anytime' 'approaching' 'away'
 'beat' 'beaten' 'called' 'caps' 'cause' 'checks' 'chest' 'come'
 'comments' 'correctly' 'crap' 'debut' 'defenseman' 'deserve' 'devils'
 'does' 'doesn' 'enforcers' 'enjoy' 'ethics' 'fight' 'fights' 'foremost'
 'game' 'games' 'general' 'gets' 'goal' 'goalie' 'got' 'hard' 'hockey'
 'holding' 'honest' 'ice' 'infinity' 'interesting' 'islanders' 'isles'
 'know' 'lackluster' 'late' 'lenient' 'like' 'lines' 'looked' 'losing'
 'lost' 'mean' 'meet' 'mentioned' 'minus' 'misplayed' 'need' 'needless'
 'negative' 'night' 'noticed' 'open' 'opinion' 'ot' 'penalties' 'pens'
 'performances' 'period' 'physical' 'pilon' 'place' 'play' 'played'
 'playoffs' 'planet' 'plus' 'pretty' 'prevent' 'prowess' 'pull'
 'ramblings' 'really' 'referee' 'referees' 'run' 'score' 'scoring'
 'seeing' 'shame' 'shoot' 'shot' 'sick' 'slap' 'slow' 'soon' 'start'
 'starts' 'stolen' 'strategy' 'subject' 'suck' 'takes' 'talent' '

 Por último vamos a analizar un mensaje recuperado con la consulta 61(también de hockey).
 Este mensaje, el 3168 ha sido recuperado usando tf/idf mientras que con frecuencias no se había recuperado:

In [85]:
write_terms(feature_names,None,test_vector_data,61)

Mensaje 61 vectorizado: ['astonished' 'attitude' 'audience' 'behaviour' 'champs' 'coins' 'crowd'
 'czechs' 'didn' 'extra' 'game' 'gave' 'germans' 'golf' 'got' 'guess'
 'help' 'kill' 'lines' 'lunatic' 'mean' 'newsreader' 'organization'
 'penalty' 'penalties' 'period' 'pisses' 'possible' 'pucks' 'really'
 'referee' 'remember' 'rink' 'single' 'soccer' 'sport' 'stupid' 'subject'
 'sucks' 'team' 'threw' 'tin' 'today' 'trash' 'version' 'watched' 'went'] 



In [89]:
write_terms(feature_names,None,train_vector_data,3618)
print("\n")
#Descomentar linea de abajo para ver el mensaje
print(train_data.data[3618])

Mensaje 3618 vectorizado: ['article' 'check' 'computer' 'courses' 'dept' 'golf' 'header' 'hockey'
 'interviews' 'kielbasa' 'lines' 'metropolitan' 'newsgroup' 'noose'
 'organization' 'phil' 'posting' 'rainbow' 'roam' 'science' 'sport'
 'subject' 'tee' 'times' 'woops' 'writes'] 



From: lee139@gaul.csd.uwo.ca (Steve Lee)
Subject: Re: LIST OF TEE TIMES AT METROPOLITAN TORONTO GOLF COURSES FOR MONDAY
Organization: Computer Science Dept., Univ. of Western Ontario, London, Canada
Nntp-Posting-Host: obelix.gaul.csd.uwo.ca
Lines: 18


In article <C5spEC.EuC@noose.ecn.purdue.edu> stamber@rainbow.ecn.purdue.edu (Kevin L. Stamber) writes:

>Kevin L. Stamber
>Purdue University
>...and Phil Kirzyc (The Kielbasa Kid) will roam the Arena for interviews.


Woops!  This is rec.sport.hockey! Not rec.sport.golf!  Hope you check the
newsgroup header next time before posting!

Steve

-------------------------------------------------------------------------------
Steve Lee      *      University of Western

Vemos que este mensaje ha sido recuperado gracias a un pequeño peso que tienen algunas palabras que frecuentan en otros documentos; y un gran peso a palabras como "sport" y "hockey". Aunque este mensaje hay que estudiarle mejor, por que vemos que el contenido del mensaje era sobre golf, pero el usuario se equivocó y en vez de en rec.sport.golf lo comentó en rec.sport.hockey.

Por lo tanto, si hubieramos hecho una consulta con algun mensaje de la clase golf(la cual no existe en nuestro conjunto de datos), seguramente habríamos obtenido este mensaje también.

Nos lo recupera aunque no sea de hockey, solo por el hecho de que el usuario comenta "Woops!  This is rec.sport.hockey!", y la palabra "hockey" tiene un gran peso a la hora de recuperar documentos parecidos a este mensaje.